In [1]:
import sys
import pyspark
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName("la3")\
        .getOrCreate()
sc=pyspark.SparkContext.getOrCreate()

In [2]:
import pandas as pd

ratings  = pd.read_csv('./imdb/ratings.csv',low_memory=False)

In [3]:
user_ids=ratings.drop_duplicates('userId')['userId']

In [4]:
target_user=int(user_ids.sample(1))

# take the rating info
user_ratings=ratings.loc[ratings['userId']==target_user]
train=user_ratings.sample(int(len(user_ratings)*0.8))
test=user_ratings[~user_ratings.movieId.isin(train.movieId)]



In [145]:
train_movies=train.drop_duplicates('movieId')
test_movies=test.drop_duplicates('movieId')

train_users=ratings.merge(movies_train,left_on='movieId',right_on='movieId').drop_duplicates('userId_x')
test_users=ratings.merge(movies_test,left_on='movieId',right_on='movieId').drop_duplicates('userId_x')
common_users=train_users.merge(test_users,left_on='userId_x',right_on='userId_x').drop_duplicates('userId_x')

train_rating=ratings.merge(movies_train,left_on='movieId',right_on='movieId',how='right').merge(common_users,left_on='userId_x',right_on='userId_x')[['userId_x','movieId','rating','timestamp']]


In [198]:
import math
train['rating2']=train['rating']**2
def cosine(movies):
    common_vector=train.merge(movies,left_on='movieId',right_on='movieId',how='left').fillna(0)
    common_vector['rating_y2']=common_vector['rating_y']**2
    common_vector['rating_xy']=common_vector['rating_x']*common_vector['rating_y']
    return common_vector.rating_xy.sum()/(math.sqrt(common_vector.rating2.sum() * common_vector.rating_y2.sum()))
sim_matrix=train_rating.groupby(['userId_x'])[['movieId','rating']].apply(cosine)


userId_x
229       0.818748
346       0.819811
611       0.844116
682       0.805004
741       0.855204
1149      0.809146
1757      0.835710
2506      0.836970
2531      0.867686
3430      0.800344
3437      0.884262
3778      0.900982
3910      0.805043
3967      0.810613
4073      0.874676
4160      0.805649
4294      0.852136
4323      0.854462
4785      0.831637
5102      0.846914
5378      0.848924
5512      0.823414
5829      0.906085
5915      0.809480
6513      0.828940
7002      0.803964
7040      0.823263
7158      0.818018
7238      0.830467
7417      0.819171
            ...   
262769    0.867415
262982    0.835940
263081    0.849750
263115    0.830458
263265    0.885169
263870    0.811794
263907    0.865037
264848    0.898239
264874    0.884591
265067    0.851461
265270    0.812310
265327    0.840131
265840    0.819125
265926    0.805149
266235    0.804364
266243    0.820008
266783    0.856342
267131    0.856968
267597    0.829956
267772    0.916205
268239    0.837845
268

In [286]:
def getRate(m):
    e_m=m.loc['movieId']
    similarity=pd.DataFrame({'userId':list(sim_matrix.keys()),'sim':list(sim_matrix.values)})
    #similarity=similarity[similarity.userId!=target_user]
    similarity=similarity.merge(ratings[ratings['movieId']==e_m],left_on='userId',right_on='userId')[['sim','userId','rating']]
    similarity['simR']=similarity['sim']*similarity['rating']
    return [e_m,m.loc['rating'],similarity['simR'].sum()/similarity['sim'].sum()]

test.apply(getRate,axis=1)

17097360     [780.0, 4.0, 3.2906008011629866]
17097362     [912.0, 4.0, 4.2470189926426505]
17097364     [1077.0, 4.0, 3.862227302639372]
17097365    [1084.0, 4.0, 3.9062767506383285]
17097366     [1179.0, 4.0, 3.720318244336796]
17097372    [1234.0, 5.0, 4.1598320580894494]
17097379     [1517.0, 4.0, 3.424552170781906]
17097385     [1610.0, 4.0, 3.841302040884119]
dtype: object

In [261]:
test

,userId,movieId,rating,timestamp
17097360,177513,780,4.0,891165098
17097362,177513,912,4.0,891556853
17097364,177513,1077,4.0,891556853
17097365,177513,1084,4.0,891556853
17097366,177513,1179,4.0,891556853
17097372,177513,1234,5.0,891556666
17097379,177513,1517,4.0,891469337
17097385,177513,1610,4.0,891469251


In [93]:
m_list=[]
movies=test
for u_movie in test['movieId']:
    m_list.append(float(movies.loc[u_movie==movies['movieId']]['rating']) if u_movie in list(movies['movieId']) else 0)
m_list

[4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0]